In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: unset; }</style>"))

In [ ]:
import os
import sys
import cv2
import numpy as np

# _/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/
#  MAIN ROUTINE OF LS-EYE
# _/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/
from autoscience.featureHandler import FeatureHandler
from autoscience.tomlHandler import TomlHandler
from autoscience.imageSegmentHadler import *
from autoscience.imageHandler import *

if __name__ == '__main__':
    #===================================================================================
    # [01] Option Handling 
    tomlObj        = TomlHandler.getTomlObject(fileNameToml="input.toml")
    fileNameImage  = tomlObj['InputFile']['fileNameImage']
    caseName       = os.path.splitext(fileNameImage)[0]
    
    #===================================================================================
    # [02] Read Input Image File
    img = ImageHandler.Open(fileNameImage)
    ImageSegmentationHandler.displayImageMatplotLib(img)

    #===================================================================================
    # [03] Read Input Image File
    #img     = ImageSegmentationHandler.executeColoringByKmeans(img)
    #ImageSegmentationHandler.displayImageCV(img)

    if not('segments' in tomlObj['RestartFile']):
        #===================================================================================
        # [04] Image Segmentation 
        # https://scikit-image.org/docs/dev/api/skimage.segmentation.html
        segments = ImageSegmentationHandler.executeSegmentation(img,modeSegmentation=SegmentationType.SKIMAGE_SLIC,options=[500,10,1,1])
        #segments = ImageSegmentationHandler.executeSegmentationGuidedFilterBased(img,caseName=os.path.splitext(fileNameImage)[0],withSharpening=True,withDisplayImages=True)
        
        tomlDict = TomlHandler.getNewTomlForApp(appName="AutoScience",version="1.0")
        tomlDict['InputFile']   = { 'fileNameImage': fileNameImage }
        tomlDict['RestartFile'] = { 'segments': segments.tolist()  }
        TomlHandler.save(tomlDict,fileNameToml="restart.toml")
    else:
        segments = np.array(tomlObj['RestartFile']['segments'])
    
    #===================================================================================
    # [05] Region Adjacency Graphs (RAGs) and merge segments based on RAG
    # https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_rag_draw.html
    #rag      = ImageSegmentationHandler.getRAG(img,segments)
    #segments = ImageSegmentationHandler.mergeSegmentsByRAG(segments,rag,option=[10,False,True])
    #segments = segments + 1
    #ImageSegmentationHandler.displaySegmentsInfo(segments)

    #===================================================================================
    # [07] Handling Statistics of Image Segments
    flagGetImageSegmentStatistics = True
    if flagGetImageSegmentStatistics:
        numSegment                      = np.max(segments)
        numSeg, label, stats, centroids = ImageSegmentationHandler.getStatisticsOfSegments(segments)
        bndToSegLabel                   = ImageSegmentationHandler.getNetworkFromImageSegments(segments)
    
        print("######################:"+str(len(centroids)))

        imgResult, resultSet, dfGLCM, isgMap = FeatureHandler.extractFeature(numSeg, label, stats, img,
                                                    numPixelScreening=10,
                                                    fileNameImageSegmented=caseName+'-Segmented.png',
                                                    fileNameImageGLCM=caseName+'-GLCM.png',
                                                    withDisplayImages=False,
                                                    withScatterPlot=False)
    
        varNameForMarkerColor = 'contrast'
        varNameList           = ['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'energy', 'correlation', 'area']
        title                 = "Scatterplot Matrix (SPLOM) for Auto-Science"
        fig = PlotlyHandler.makePairPlot(df=dfGLCM,varNameList=varNameList,varNameForMarkerColor=varNameForMarkerColor,width=1000,height=1000,title=title)
        fig.show()

    #===================================================================================
    # [08] Execute Coloring of Segments and display
    imgColored = ImageSegmentationHandler.executeColoringOfSegments(img,segments,option=['avg'],withMarkBoundaries=False)
    ImageSegmentationHandler.displayImageMatplotLib(imgColored)
    
    #===================================================================================
    # [09] Do something interesting for each segments
    #for isg in range(numSegment):
    #    if stats[isg][4]>100:
    #        imgLoc, mask = ImageSegmentationHandler.getLocalImageOfImageSegment(img=img,segments=segments,label_target=isg+1)
    #        ImageSegmentationHandler.displayImageMatplotLib(imgLoc)

    #======================================================================
    # [10] RAG (Region Adjacency Graph) Handling 
    # ImageSegmentationHandler.test_displaySegmentNetworkByRagMeanColor(img,segments,centroids)

    #======================================================================
    # [11] Mark Segment Boundaries and display
    imgMarkSegBnd = ImageSegmentationHandler.markBoudariesOfSegments(img,segments)
    ImageSegmentationHandler.displayImageMatplotLib(imgMarkSegBnd)
    fileNameImageResult  = caseName+'-imgResult.png'
    cv2.imwrite(fileNameImageResult, imgMarkSegBnd)
    ImageSegmentationHandler.displayNetworkOnImage(img=imgMarkSegBnd,segments=segments,centroids=centroids,bndToSegLabel=bndToSegLabel)

In [3]:
label_rgb = skimage.color.label2rgb(segments, img, kind='avg')

In [ ]:
from autoscience.colorHandler import *

neighborSegmentList = ImageSegmentationHandler.generateNeightborSegmentList(numSegment=numSegment,bndToSegLabel=bndToSegLabel)

flagSegmentVisited  = [False for isg in range(numSeg)]

isg_base      = 173
colorAve_base = ImageSegmentationHandler.getMeanColorOfImageSegment(img=img,stats=stats,segments=label,isg_target=isg_base)

isgList  = [isg_base]
for ii in range(len(isgList)):
    isg = isgList[ii]
    flagSegmentVisited[isg] = True

colDist_cri_base = 0.120
colDist_cri      = 0.055

numCallMax  = 10
numCall     = 0

while len(isgList) > 0 and numCall<numCallMax: 

    print("######################### numCall:"+str(numCall))

    isgListNew = []
    for ii in range(len(isgList)):
        isg = isgList[ii]

        print("isg:"+str(isg))
        colorAve = ImageSegmentationHandler.getMeanColorOfImageSegment(img=img,stats=stats,segments=label,isg_target=isg)

        for ii in range(len(neighborSegmentList[isg])):
            isg_n        = neighborSegmentList[isg][ii]
            colorAve_n   = ImageSegmentationHandler.getMeanColorOfImageSegment(img=img,stats=stats,segments=label,isg_target=isg_n)
            colDist_base = ColorDistance.getColorEuclideanDistanceOnRGBSystemNormalized(color1=colorAve_base,color2=colorAve_n)
            colDist      = ColorDistance.getColorEuclideanDistanceOnRGBSystemNormalized(color1=colorAve,color2=colorAve_n)

            if colDist<colDist_cri and colDist_base<colDist_cri_base and flagSegmentVisited[isg_n]==False:
                flagSegmentVisited[isg_n] = True
                isgListNew.append(isg_n)

            print("isg_n:"+str(isg_n)+" colDist:"+str(colDist))

    isgList = isgListNew
    numCall = numCall + 1

isgList = [n for n, v in enumerate(flagSegmentVisited) if v == True]
print(isgList)

In [ ]:
import skimage.segmentation
from matplotlib.lines import Line2D
fig = plt.figure(figsize=[20,8.4])
ax  = fig.add_subplot(111)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
plt.scatter(centroids[:,0],centroids[:,1], c='blue')
for icnt in range(bndToSegLabel.shape[1]):
    isg1  = bndToSegLabel[0,icnt]-1
    isg2  = bndToSegLabel[1,icnt]-1
    x0,y0 = centroids[isg1]
    x1,y1 = centroids[isg2]
    ax.add_line(Line2D([x0,x1],[y0,y1], alpha=0.5))

xList = []
yList = []
for ii in range(len(isgList)):
    isg = isgList[ii]
    x,y = centroids[isg]
    xList.append(x)
    yList.append(y)
plt.scatter(xList,yList, c='red')

plt.show()

In [ ]:
#==========================================================================
# TODO: Under construction
#==========================================================================

from autoscience.imageHandler import *

tomlObj        = TomlHandler.getTomlObject(fileNameToml="input.toml")
fileNameImage  = tomlObj['InputFile']['fileNameImage']
img            = ImageHandler.Open(fileNameImage)

isg_target = 238

imgPixList = ImageSegmentationHandler.getLocalImagePixelListOfImageSegment(img=img,stats=stats,segments=label,isg_target=isg_target,colorR=255,colorG=0,colorB=0,withDisplayImage=True)

B = imgPixList[:,0]
G = imgPixList[:,1]
R = imgPixList[:,2]
Y = 0.299 * R + 0.587 * G + 0.114 * B
Y = Y.astype(np.int32)

import numpy as np
import matplotlib.pyplot as plt
plt.hist(Y,40)

In [ ]:
#----------------------------------------------------------------------
# TODO: Examination for Kmean Clustering [Done]
# TODO: Implementation of color list sorting [Done]
#----------------------------------------------------------------------

from autoscience.colorHandler import *

imgRes = ImageSegmentationHandler.executeColoringByKmeans(img=img,numberOfColors=5,withColorConvert=False)

# Generate Color RGB List
from PIL import Image
PIL_image = Image.fromarray(np.uint8(imgRes)).convert('RGB')
colorList = PIL_image.getcolors()

iimax = len(colorList)
colorRGBList = []
for ii in range(iimax):
    colorRGB = (colorList[ii][1][0],colorList[ii][1][1],colorList[ii][1][2])
    colorRGBList.append(colorRGB)

# Sort Color RGB List
colorRGBList_sorted = ColorSort.sortColorListByColorEuclideanDistanceOnRGBSystem(colorRGBList=colorRGBList,colorBase=[255,255,255])

# Coloring
for ii in range(len(colorRGBList_sorted)):
    target_color = colorRGBList_sorted[ii]
    change_color = Color.getColorJET(v=ii,vmin=0,vmax=iimax-1)
    imgRes[np.where((imgRes == target_color).all(axis=2))] = change_color

PIL_image = Image.fromarray(np.uint8(imgRes)).convert('RGB')
PIL_image.save(caseName+'-imgKmeans.png')

imgRes = cv2.cvtColor(imgRes, cv2.COLOR_BGR2RGB)
ImageSegmentationHandler.displayImageMatplotLib(imgMarkSegBnd)
ImageSegmentationHandler.displayImageMatplotLib(imgRes)

In [ ]:
#----------------------------------------------------------------------
# TODO: Display Segment Mean Color as Scatter3D [Done]
#----------------------------------------------------------------------

iimax = len(isgMap)

from autoscience.tableHandler import *
df = TableHandler.getInitializedPandasTable( numColumn=3, numRow=iimax, colNames=["Segment Color R[-]","Segment Color G[-]", "Segment Color B[-]"] )

imgTemp = imgColored.copy()

for ii in range(iimax):
    isg          = isgMap[ii]
    imgLoc       = imgTemp[label==isg]
    colorAve     = imgLoc[0]
    df.iat[ii,0] = colorAve[2]
    df.iat[ii,1] = colorAve[1]
    df.iat[ii,2] = colorAve[0]

ImageSegmentationHandler.displayImageMatplotLib(imgTemp)

import plotly.express as px

trace=go.Scatter3d( x=df["Segment Color R[-]"],
                    y=df["Segment Color G[-]"],
                    z=df["Segment Color B[-]"],
                    mode='markers',
                    marker=dict(size=5,
                                color=['rgb({},{},{})'.format(r,g,b) for r,g,b in zip(df["Segment Color R[-]"].values, df["Segment Color G[-]"].values, df["Segment Color B[-]"].values)]))

data = [trace]

layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))

fig    = go.Figure(data=data, layout=layout)
fig.update_layout( width=800, height=800 )
fig.update_layout(scene_aspectmode='cube')
fig.show()

In [ ]:
from autoscience.colorHandler import *

def coloringBasedOnGLCM(img=None,label=None,isgMap=None,dfGLCM=None,nameGLCM=None):
    segMax = np.max(label)
    for isg in range(segMax):
        if not(isg in isgMap):
            img[label==isg] = (255,255,255)

    vmin = np.min(dfGLCM[nameGLCM])
    vmax = np.max(dfGLCM[nameGLCM])
    for ii in range(len(isgMap)):
        isg = isgMap[ii]
        col = Color.getColorJET(v=dfGLCM[nameGLCM].iloc[ii],vmin=vmin,vmax=vmax)
        img[label==isg] = col

    return img, vmin, vmax

def coloringBasedOnTwoGLCMs(img=None,label=None,isgMap=None,dfGLCM=None,nameGLCM1=None,nameGLCM2=None):
    segMax = np.max(label)
    for isg in range(segMax):
        if not(isg in isgMap):
            img[label==isg] = (255,255,255)
    
    vmin1 = np.min(dfGLCM[nameGLCM1])
    vmax1 = np.max(dfGLCM[nameGLCM1])
    vmin2 = np.min(dfGLCM[nameGLCM2])
    vmax2 = np.max(dfGLCM[nameGLCM2])

    omega = 0.5

    for ii in range(len(isgMap)):
        isg = isgMap[ii]
        
        v1    = (vmax1-dfGLCM[nameGLCM1].iloc[ii])/(vmax1-vmin1)
        v2    = (vmax2-dfGLCM[nameGLCM2].iloc[ii])/(vmax2-vmin2)
        col   = Color.getColorFor2Vars(v1=v1,v1min=0.0,v1max=1.0,v2=v2,v2min=0.0,v2max=1.0)
        
        img[label==isg] = col

    return img, vmin, vmax

# Segment Screening
isgMapNew = []
iiListDrop= []
for ii in range(len(isgMap)):
    isg     = isgMap[ii]
    valGLCM = dfGLCM['energy'][ii]
    if valGLCM<0.07:
        isgMapNew.append(isg)
    else:
        iiListDrop.append(ii)

dfGLCM_tmp = dfGLCM.copy()
dfGLCM_tmp.drop(index=iiListDrop, inplace=True)
print(dfGLCM_tmp)

nameListGLCM = ['contrast','dissimilarity','homogeneity','ASM','energy','correlation']
imgColored   = ImageSegmentationHandler.executeColoringOfSegments(img,segments,option=['avg'])

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(30,15))

fig.add_subplot(2,4,1)
plt.title("Original")
plt.imshow(cv2.cvtColor(imgMarkSegBnd, cv2.COLOR_RGB2BGR))

for i in range(len(nameListGLCM)):
    nameGLCM = nameListGLCM[i]
    imgColored, vmin, vmax = coloringBasedOnGLCM(img=imgColored,label=label,isgMap=isgMapNew,dfGLCM=dfGLCM_tmp,nameGLCM=nameGLCM)
    fig.add_subplot(2,4,i+2)
    plt.title(nameGLCM+" ["+str('{:f}'.format(vmin))+", "+str('{:f}'.format(vmax))+"]")
    plt.imshow(imgColored)

nameGLCM1 = "dissimilarity"
nameGLCM2 = "correlation"

imgColored, vmin, vmax = coloringBasedOnTwoGLCMs(img=imgColored,label=label,isgMap=isgMapNew,dfGLCM=dfGLCM_tmp,nameGLCM1=nameGLCM1,nameGLCM2=nameGLCM2)
fig.add_subplot(2,4,8)
plt.title(nameGLCM1+"/"+nameGLCM2+" ["+str('{:f}'.format(vmin))+", "+str('{:f}'.format(vmax))+"]")
plt.imshow(imgColored)

plt.show()
dfGLCM

#----------------------------------------------------------------------
# TODO: Display Color Bar Range Values [Done]
# TODO: Adjustment of Color Bar Range [Pending]
# TODO: Comparison of multiple images [Pending]
# TODO: Implementation of the other type of GLCM [Pending]
#----------------------------------------------------------------------

In [ ]:
segmentsOrg   = label
fileNameImage = tomlObj['InputFile']['fileNameImage']
img           = cv2.imread(fileNameImage, cv2.IMREAD_COLOR)

import skimage
from skimage.segmentation import *
from   skimage import img_as_ubyte

from IPython.display import clear_output

def update_point(trace, points, selector):
    segmentList = list(points.point_inds)
    with fig.batch_update():
        OnUpdate(centroids,stats,segmentsOrg,img,segmentList=segmentList)

import plotly.graph_objs as go
import pandas as pd
from autoscience.plotlyHandler import *

def OnUpdate(centroids,stats,segmentsOrg,img,segmentList=None):
    global scatter, fig, update_point, OnSelection
    clear_output(wait=True)
    
    df = pd.read_csv('tabGLCM.csv')
    #print(df)
    varNameForMarkerColor = 'contrast'
    varNameList           = ['contrast', 'dissimilarity', 'homogeneity', 'asm', 'energy', 'correlation', 'area']
    title                 = "Scatterplot Matrix (SPLOM) for Auto-Science"
    fig = PlotlyHandler.makePairPlot(df=df,varNameList=varNameList,varNameForMarkerColor=varNameForMarkerColor,width=1000,height=1000,title=title)
    fig.show()
    
    scatter              = fig.data[0]
    colors               = ['#a3a7e4'] * 100
    scatter.marker.color = colors
    scatter.marker.size  = [10] * 100
    fig.layout.hovermode   = 'closest'

    scatter.on_selection(update_point)
    #scatter.on_click(update_point)
    
    fig.open()
    display(fig)
 
    segments = segmentsOrg.copy()
    if segmentList!=None:
        numSeg = np.max(segments)
        for isg in range(numSeg):
            if isg not in segmentList:
                iwhere = (segments==(isg+1))
                segments[iwhere] = 0

    imgMarkSegBnd = skimage.segmentation.mark_boundaries(img, segments)
    imgMarkSegBnd = cv2.cvtColor(img_as_ubyte(imgMarkSegBnd), cv2.COLOR_RGB2BGR)

    import plotly.express as px
    from skimage import data
    fig = px.imshow(imgMarkSegBnd, color_continuous_scale='gray')
    fig.update_layout(coloraxis_showscale=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()

OnUpdate(centroids,stats,segmentsOrg,img,segmentList=None)

In [20]:
import plotly.graph_objs as go
from PIL import Image

def PlotlyMakePlotWithImageBackground(x=None,y=None,fileNameImage=None,winSizeRatio=1.0):
    text   = np.linspace(0,len(x),len(x)+1)
    data   = go.Scatter(x=x, y=y, mode="markers",text=text,hovertemplate='<br>x:%{x}<br>y:%{y}<br>Segment:%{text}')
    img    = Image.open(fileNameImage)
    width, height = img.size

    print("width:"+str(width))
    print("height:"+str(height))
    
    button1 = dict(label='Run1', method='update')
    button2 = dict(label='Run2', method='update')
    
    buttons = [button1,button2]
    
    updatemenus = [dict(active=1, type='buttons', direction='right', x=0.5, y=1.01, xanchor='center', yanchor='bottom', buttons=buttons)]
    
    layout = go.Layout(images=[dict( source=img, xref="x", yref="y", x=0, y=height,
                sizex=width, sizey=height, sizing="stretch", opacity=1.0,layer="below")],autosize = False,updatemenus=updatemenus)
    
    fig = go.FigureWidget(data=data, layout=layout)

    fig.update_yaxes (scaleanchor = "x", scaleratio=1 )
    fig.update_layout(xaxis_range=[0,width])
    fig.update_layout(yaxis_range=[0,height])
    fig.update_layout(autosize=False,width=width*winSizeRatio,height=height*winSizeRatio)

    return fig


In [21]:
from autoscience.tableHandler import *
dfGLCM_Normalized = TableHandler.getNormalizedDataFrame(tab=dfGLCM)

In [22]:
#============================================================
# TODO: Implementation of Segmented Image Analyzer [Done]
# TODO: Improvement of plot update without reopen  []
#============================================================

from autoscience.imageHandler import *

width, height = ImageHandler.getImageSize(fileNameImage=fileNameImageResult)

x = centroids[:,0]
y = np.zeros(len(centroids))
for isg in range(len(centroids)):
    y[isg] = height-centroids[isg,1]

segmentList = []
figPlot = None
def update_point(trace, points, selector):
    global scatter, fig, segmentList, figPlot

    clear_output(wait=True)
    OnUpdateLoc(x=x,y=y,fileNameImageResult=fileNameImageResult)

    with fig.batch_update():
        c = list(scatter.marker.color)
        s = list(scatter.marker.size)
    
        for isg in points.point_inds:

            if not(isg in segmentList):
                segmentList.append(isg)
        
            print("segmentList:"+str(segmentList))

            for ii in range(len(segmentList)):
                isg = segmentList[ii]
                c[isg] = '#bae2be'
                s[isg] = 10

            with fig.batch_update():
                scatter.marker.color = c
                scatter.marker.size  = s

            if len(segmentList)>0:
                colNames = ['index','contrast','dissimilarity','homogeneity','ASM','energy','correlation','area']
                df       = TableHandler.getInitializedPandasTable(numColumn=8,numRow=len(segmentList),colNames=colNames)
                
                for ii in range(len(segmentList)):
                    isg   = segmentList[ii]
                    index = isgMap.tolist().index(isg)
                    df['index'][ii] = ii
                    for jj in range(1,len(colNames)):
                        df[colNames[jj]][ii] = dfGLCM_Normalized[colNames[jj]][index]
            
                import plotly.graph_objects as go
            
                if figPlot is None:
                    figPlot = go.Figure()
                else:
                    figPlot.data   = []
                    figPlot.layout = {}
                
                for icol in range(1,len(colNames)-1): # Area is intentionally excluded
                    colName = colNames[icol]
                    figPlot.add_trace(go.Scatter(x=df['index'], y=df[colName], name=colName))

                figPlot.update_layout(legend_title='<b> GLCM (Normalized) </b>') 
                figPlot.show()
                #print(df)
                del df

icount = 0
def OnUpdateLoc(x=None,y=None,fileNameImageResult=None):
    global scatter, fig, update_point, icount, segmentList
    print("Clear Output is called in OnUpdateLoc:"+str(icount))
    icount = icount + 1
    clear_output(wait=True)

    fig                  = PlotlyMakePlotWithImageBackground(x=x,y=y,fileNameImage=fileNameImageResult,winSizeRatio=0.5)
    scatter              = fig.data[0]
    n                    = numSeg
    scatter.marker.color = ['blue'] * n
    scatter.marker.size  = [5]      * n

    print("segmentList in OnUpdateLoc:"+str(segmentList))

    fig.layout.hovermode = 'closest'
    scatter.on_click(update_point)
    display(fig)

OnUpdateLoc(x=x,y=y,fileNameImageResult=fileNameImageResult)

width:2304
height:1728
segmentList in OnUpdateLoc:[]


FigureWidget({
    'data': [{'hovertemplate': '<br>x:%{x}<br>y:%{y}<br>Segment:%{text}',
              'marker…

In [ ]:
#============================================================
# TODO: Implementation of Property Polar Plot [Done]
#============================================================

import pandas as pd
from autoscience.plotlyHandler import *

dfTarget  = dfGLCM_Normalized
numColumn = len(dfTarget.columns)-1
numRow    = len(segmentList)

df = TableHandler.getInitializedPandasTable(numColumn=numColumn,numRow=numRow,colNames=list(dfTarget.columns))

indexList = []
for irow in range(numRow):
    isg   = segmentList[irow]
    indexList.append("isg:"+str(isg))

df.index = indexList

for irow in range(numRow):
    isg   = segmentList[irow]
    index = isgMap.tolist().index(isg)
    prop  = list(dfTarget.iloc[index])[:-1]
    for icol in range(numColumn):
        df.iloc[irow,icol] = prop[icol]

PlotlyHandler.makePropertyPolarPlot(df=df,fillType='none',RadialAxisVisible=False,displayOnly=True)

In [24]:
condition = (dfGLCM["contrast"]>200) & (dfGLCM["dissimilarity"]>20.0)

df_ext = dfGLCM[condition]
print(df_ext)

indexList = list(df_ext.index)

segmentList_ext = []
for ii in range(len(indexList)):
    index = indexList[ii]
    isg   = isgMap[index]
    segmentList_ext.append(isg)

print(segmentList_ext)

Empty DataFrame
Columns: [contrast, dissimilarity, homogeneity, ASM, energy, correlation, area]
Index: []
[]


In [ ]:
print("segmentList:"+str(segmentList))

segments = segmentsOrg.copy()

ratioImageDisplay = 0.5

if segmentList!=None:
    numSeg = np.max(segments)
    for isg in range(numSeg+1):
        if isg not in segmentList_ext:
            iwhere = (segments==isg)
            segments[iwhere] = 0

    imgMarkSegBnd = skimage.segmentation.mark_boundaries(img, segments)
    imgMarkSegBnd = cv2.cvtColor(img_as_ubyte(imgMarkSegBnd), cv2.COLOR_RGB2BGR)

    height = imgMarkSegBnd.shape[0]
    width  = imgMarkSegBnd.shape[1]

    import plotly.express as px
    from skimage import data
    fig = px.imshow(imgMarkSegBnd, color_continuous_scale='gray')
    fig.update_layout(coloraxis_showscale=False,width=width*ratioImageDisplay,height=height*ratioImageDisplay)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()